In [1]:
from bs4 import BeautifulSoup
import requests
import time
import unicodedata
import re

def transformaFecha(fecha):
    '''
        Transforma la fecha del formato (ejemplo): "Resultado del  jueves 17 de agosto de 2017"
        a AAAAMMDD (20170817 en el ejemplo anterior)
        
        Argumentos: fecha --> string en formato "Resultado del  jueves 17 de agosto de 2017"
        Valor devuelto: fechaT --> fecha transformada al formato AAAAMMDD (string)
    '''
    
    meses = {'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06',
            'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre':'12'}
    
    subcadenas = fecha.split()
    anio = subcadenas[-1]
    mes = meses[subcadenas[-3]] 
    if len(subcadenas[-5]) == 1: 
        dia = '0' + subcadenas[-5]
    else:
        dia = subcadenas[-5]
    fechaT = anio + mes + dia
    return fechaT

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:82.0) Gecko/20100101 Firefox/82.0'}

fecha_desde = '20/06/1991' # Fecha de inicio del histórico. La más antigua es 20/06/1991
fecha_hasta = '31/10/2020' # Fecha fin del histórico. La más reciente ha de ser el dia de hoy
pagina = 'https://www.loteriasyapuestas.es/es/buscador?startDate=' + fecha_desde + '&gameId=04&type=search&endDate=' + fecha_hasta
siguiente = True
indice = 0

# Grabar la cabecera en el fichero .csv
cabecera = "Indice;Fecha_larga;Fecha_AAAAMMDD;B1;B2;B3;B4;B5;B6;C;R;Recaudación;Bote"
cabecera = cabecera + ";ACE;PCE;AC1;PC1;AC2;PC2;AC3;PC3;AC4;PC4;AC5;PC5;ACR;PCR"
fic_historico = open("Historico_Primitiva.csv", "w")   
fic_historico.write(cabecera)
fic_historico.write("\n")

combinacion_ganadora = [] # Inicializa la lista donde irán los datos

while siguiente:
    
    # Recoger los datos de los sorteos de la página
    page = requests.get(pagina, headers=headers)
    soup = BeautifulSoup(page.content, features='lxml')
    sorteos = soup.find_all('div',class_='textoCuerpo buscadorAvanzadoFuturos primitiva cuerpoRegionInf celebrado')
    
    #print(sorteos)
    
    for elemento in sorteos:
        indice += 1
        combinacion_ganadora.append(str(indice))
    
        # Fecha larga en letras
        fecha=elemento.h3
        combinacion_ganadora.append(unicodedata.normalize("NFKD",fecha.string)) # Para quitar \xa0
    
        # Transforma la fecha larga a un formato AAAAMMDD
        fecha = transformaFecha(str(fecha.string))
        combinacion_ganadora.append(fecha)
    
        # Capturamos las combinaciones del sorteo
        combinacion = elemento.find_all('li')
        for bola in combinacion:
            # El reintegro y complementario están dentro de un span
            if bola.span:
                combinacion_ganadora.append(bola.span.string)
            else:
                combinacion_ganadora.append(bola.string)
                
        # Más información: accedemos a la página para extraer información adicional
        mas_info = "https://www.loteriasyapuestas.es" + elemento.find(class_='enlace').find('a')['href']
        mas_info_page = requests.get(mas_info, headers=headers)
        mas_info_soup= BeautifulSoup(mas_info_page.content, features='lxml')
        #print(mas_info_soup)
        
        # Importe de la recaudación
        recaudacion = mas_info_soup.find(string=re.compile("Recaudación:"))
        #print(recaudacion)
        if recaudacion:  # Nota: al ejecutar el script dio error en ciertas fechas por falta de datos
            combinacion_ganadora.append(recaudacion.split()[-2]) 
        else:
            combinacion_ganadora.append("None") # Si la recaudación no aparece, guardamos 'None'
        
        # Importe del bote
        bote = mas_info_soup.find(string=re.compile("Bote ofrecido:"))
        if bote: # Nota: al ejecutar el script dio error en ciertas fechas por falta de datos
            combinacion_ganadora.append(bote.split()[-2]) 
        else:
            combinacion_ganadora.append("None") # Si el bote no aparece, guardamos 'None'
            
        # Tabla de premios
        tabla_premios = mas_info_soup.table.find_all(class_="izquierda")
        for linea in tabla_premios:
            # Número de acertantes
            num_acert = linea.find_next_siblings("td")[0].string
            if num_acert:
                combinacion_ganadora.append(num_acert) 
            else:
                combinacion_ganadora.append("None")
            # Premio
            #print(linea.find_next_siblings("td")[1])
            
            valor_premio = linea.find_next_siblings("td")[1].string.split()
            if (len(valor_premio) >= 2):
                combinacion_ganadora.append(valor_premio[-2])  
            else:
                combinacion_ganadora.append("None") # Si el premio no está informado se guarda "None"                                  
    
        # Guardar en el fichero .csv
        print(combinacion_ganadora[1] + ": PROCESADO")
        #print(combinacion_ganadora)
        fic_historico.write(';'.join(combinacion_ganadora))
        fic_historico.write("\n")
        
        combinacion_ganadora = [] # Reinicializamos la linea de la combinación
        
        # Buscamos la página que se indica en el botón "ver siguiente"
        siguiente = soup.find(title='ver siguiente')
        if siguiente: pagina="https://www.loteriasyapuestas.es" + siguiente["href"]
        time.sleep(1) # Delay para no saturar el servidor
        
fic_historico.close()

 Resultado del  sábado 31 de octubre de 2020: PROCESADO
 Resultado del  jueves 29 de octubre de 2020: PROCESADO
 Resultado del  sábado 24 de octubre de 2020: PROCESADO
 Resultado del  jueves 22 de octubre de 2020: PROCESADO
 Resultado del  sábado 17 de octubre de 2020: PROCESADO
 Resultado del  jueves 15 de octubre de 2020: PROCESADO
 Resultado del  sábado 10 de octubre de 2020: PROCESADO
 Resultado del  jueves 8 de octubre de 2020: PROCESADO
 Resultado del  sábado 3 de octubre de 2020: PROCESADO
 Resultado del  jueves 1 de octubre de 2020: PROCESADO
 Resultado del  sábado 26 de septiembre de 2020: PROCESADO
 Resultado del  jueves 24 de septiembre de 2020: PROCESADO
 Resultado del  sábado 19 de septiembre de 2020: PROCESADO
 Resultado del  jueves 17 de septiembre de 2020: PROCESADO
 Resultado del  sábado 12 de septiembre de 2020: PROCESADO
 Resultado del  jueves 10 de septiembre de 2020: PROCESADO
 Resultado del  sábado 5 de septiembre de 2020: PROCESADO
 Resultado del  jueves

 Resultado del  sábado 30 de marzo de 2019: PROCESADO
 Resultado del  jueves 28 de marzo de 2019: PROCESADO
 Resultado del  sábado 23 de marzo de 2019: PROCESADO
 Resultado del  jueves 21 de marzo de 2019: PROCESADO
 Resultado del  sábado 16 de marzo de 2019: PROCESADO
 Resultado del  jueves 14 de marzo de 2019: PROCESADO
 Resultado del  sábado 9 de marzo de 2019: PROCESADO
 Resultado del  jueves 7 de marzo de 2019: PROCESADO
 Resultado del  sábado 2 de marzo de 2019: PROCESADO
 Resultado del  jueves 28 de febrero de 2019: PROCESADO
 Resultado del  sábado 23 de febrero de 2019: PROCESADO
 Resultado del  jueves 21 de febrero de 2019: PROCESADO
 Resultado del  sábado 16 de febrero de 2019: PROCESADO
 Resultado del  jueves 14 de febrero de 2019: PROCESADO
 Resultado del  sábado 9 de febrero de 2019: PROCESADO
 Resultado del  jueves 7 de febrero de 2019: PROCESADO
 Resultado del  sábado 2 de febrero de 2019: PROCESADO
 Resultado del  jueves 31 de enero de 2019: PROCESADO
 Resultad

 Resultado del  jueves 2 de noviembre de 2017: PROCESADO
 Resultado del  sábado 28 de octubre de 2017: PROCESADO
 Resultado del  jueves 26 de octubre de 2017: PROCESADO
 Resultado del  sábado 21 de octubre de 2017: PROCESADO
 Resultado del  jueves 19 de octubre de 2017: PROCESADO
 Resultado del  sábado 14 de octubre de 2017: PROCESADO
 Resultado del  jueves 12 de octubre de 2017: PROCESADO
 Resultado del  sábado 7 de octubre de 2017: PROCESADO
 Resultado del  jueves 5 de octubre de 2017: PROCESADO
 Resultado del  sábado 30 de septiembre de 2017: PROCESADO
 Resultado del  jueves 28 de septiembre de 2017: PROCESADO
 Resultado del  sábado 23 de septiembre de 2017: PROCESADO
 Resultado del  jueves 21 de septiembre de 2017: PROCESADO
 Resultado del  sábado 16 de septiembre de 2017: PROCESADO
 Resultado del  jueves 14 de septiembre de 2017: PROCESADO
 Resultado del  sábado 9 de septiembre de 2017: PROCESADO
 Resultado del  jueves 7 de septiembre de 2017: PROCESADO
 Resultado del  sá

 Resultado del  jueves 2 de junio de 2016: PROCESADO
 Resultado del  sábado 28 de mayo de 2016: PROCESADO
 Resultado del  jueves 26 de mayo de 2016: PROCESADO
 Resultado del  sábado 21 de mayo de 2016: PROCESADO
 Resultado del  jueves 19 de mayo de 2016: PROCESADO
 Resultado del  sábado 14 de mayo de 2016: PROCESADO
 Resultado del  jueves 12 de mayo de 2016: PROCESADO
 Resultado del  sábado 7 de mayo de 2016: PROCESADO
 Resultado del  jueves 5 de mayo de 2016: PROCESADO
 Resultado del  sábado 30 de abril de 2016: PROCESADO
 Resultado del  jueves 28 de abril de 2016: PROCESADO
 Resultado del  sábado 23 de abril de 2016: PROCESADO
 Resultado del  jueves 21 de abril de 2016: PROCESADO
 Resultado del  sábado 16 de abril de 2016: PROCESADO
 Resultado del  jueves 14 de abril de 2016: PROCESADO
 Resultado del  sábado 9 de abril de 2016: PROCESADO
 Resultado del  jueves 7 de abril de 2016: PROCESADO
 Resultado del  sábado 2 de abril de 2016: PROCESADO
 Resultado del  jueves 31 de marz

 Resultado del  sábado 27 de diciembre de 2014: PROCESADO
 Resultado del  jueves 25 de diciembre de 2014: PROCESADO
 Resultado del  sábado 20 de diciembre de 2014: PROCESADO
 Resultado del  jueves 18 de diciembre de 2014: PROCESADO
 Resultado del  sábado 13 de diciembre de 2014: PROCESADO
 Resultado del  jueves 11 de diciembre de 2014: PROCESADO
 Resultado del  sábado 6 de diciembre de 2014: PROCESADO
 Resultado del  jueves 4 de diciembre de 2014: PROCESADO
 Resultado del  sábado 29 de noviembre de 2014: PROCESADO
 Resultado del  jueves 27 de noviembre de 2014: PROCESADO
 Resultado del  sábado 22 de noviembre de 2014: PROCESADO
 Resultado del  jueves 20 de noviembre de 2014: PROCESADO
 Resultado del  sábado 15 de noviembre de 2014: PROCESADO
 Resultado del  jueves 13 de noviembre de 2014: PROCESADO
 Resultado del  sábado 8 de noviembre de 2014: PROCESADO
 Resultado del  jueves 6 de noviembre de 2014: PROCESADO
 Resultado del  sábado 1 de noviembre de 2014: PROCESADO
 Resultado

...

 Resultado del  jueves 25 de diciembre de 1997: PROCESADO
 Resultado del  sábado 20 de diciembre de 1997: PROCESADO
 Resultado del  jueves 18 de diciembre de 1997: PROCESADO
 Resultado del  sábado 13 de diciembre de 1997: PROCESADO
 Resultado del  jueves 11 de diciembre de 1997: PROCESADO
 Resultado del  sábado 6 de diciembre de 1997: PROCESADO
 Resultado del  jueves 4 de diciembre de 1997: PROCESADO
 Resultado del  sábado 29 de noviembre de 1997: PROCESADO
 Resultado del  jueves 27 de noviembre de 1997: PROCESADO
 Resultado del  sábado 22 de noviembre de 1997: PROCESADO
 Resultado del  jueves 20 de noviembre de 1997: PROCESADO
 Resultado del  sábado 15 de noviembre de 1997: PROCESADO
 Resultado del  jueves 13 de noviembre de 1997: PROCESADO
 Resultado del  sábado 8 de noviembre de 1997: PROCESADO
 Resultado del  jueves 6 de noviembre de 1997: PROCESADO
 Resultado del  sábado 1 de noviembre de 1997: PROCESADO
 Resultado del  jueves 30 de octubre de 1997: PROCESADO
 Resultado de

 Resultado del  sábado 27 de julio de 1996: PROCESADO
 Resultado del  jueves 25 de julio de 1996: PROCESADO
 Resultado del  sábado 20 de julio de 1996: PROCESADO
 Resultado del  jueves 18 de julio de 1996: PROCESADO
 Resultado del  sábado 13 de julio de 1996: PROCESADO
 Resultado del  jueves 11 de julio de 1996: PROCESADO
 Resultado del  sábado 6 de julio de 1996: PROCESADO
 Resultado del  jueves 4 de julio de 1996: PROCESADO
 Resultado del  sábado 29 de junio de 1996: PROCESADO
 Resultado del  jueves 27 de junio de 1996: PROCESADO
 Resultado del  sábado 22 de junio de 1996: PROCESADO
 Resultado del  jueves 20 de junio de 1996: PROCESADO
 Resultado del  sábado 15 de junio de 1996: PROCESADO
 Resultado del  jueves 13 de junio de 1996: PROCESADO
 Resultado del  sábado 8 de junio de 1996: PROCESADO
 Resultado del  jueves 6 de junio de 1996: PROCESADO
 Resultado del  sábado 1 de junio de 1996: PROCESADO
 Resultado del  jueves 30 de mayo de 1996: PROCESADO
 Resultado del  sábado 2

 Resultado del  jueves 23 de febrero de 1995: PROCESADO
 Resultado del  sábado 18 de febrero de 1995: PROCESADO
 Resultado del  jueves 16 de febrero de 1995: PROCESADO
 Resultado del  sábado 11 de febrero de 1995: PROCESADO
 Resultado del  jueves 9 de febrero de 1995: PROCESADO
 Resultado del  sábado 4 de febrero de 1995: PROCESADO
 Resultado del  jueves 2 de febrero de 1995: PROCESADO
 Resultado del  sábado 28 de enero de 1995: PROCESADO
 Resultado del  jueves 26 de enero de 1995: PROCESADO
 Resultado del  sábado 21 de enero de 1995: PROCESADO
 Resultado del  jueves 19 de enero de 1995: PROCESADO
 Resultado del  sábado 14 de enero de 1995: PROCESADO
 Resultado del  jueves 12 de enero de 1995: PROCESADO
 Resultado del  sábado 7 de enero de 1995: PROCESADO
 Resultado del  jueves 5 de enero de 1995: PROCESADO
 Resultado del  sábado 31 de diciembre de 1994: PROCESADO
 Resultado del  jueves 29 de diciembre de 1994: PROCESADO
 Resultado del  sábado 24 de diciembre de 1994: PROCESAD

 Resultado del  sábado 25 de septiembre de 1993: PROCESADO
 Resultado del  jueves 23 de septiembre de 1993: PROCESADO
 Resultado del  sábado 18 de septiembre de 1993: PROCESADO
 Resultado del  jueves 16 de septiembre de 1993: PROCESADO
 Resultado del  sábado 11 de septiembre de 1993: PROCESADO
 Resultado del  jueves 9 de septiembre de 1993: PROCESADO
 Resultado del  sábado 4 de septiembre de 1993: PROCESADO
 Resultado del  jueves 2 de septiembre de 1993: PROCESADO
 Resultado del  sábado 28 de agosto de 1993: PROCESADO
 Resultado del  jueves 26 de agosto de 1993: PROCESADO
 Resultado del  sábado 21 de agosto de 1993: PROCESADO
 Resultado del  jueves 19 de agosto de 1993: PROCESADO
 Resultado del  sábado 14 de agosto de 1993: PROCESADO
 Resultado del  jueves 12 de agosto de 1993: PROCESADO
 Resultado del  sábado 7 de agosto de 1993: PROCESADO
 Resultado del  jueves 5 de agosto de 1993: PROCESADO
 Resultado del  sábado 31 de julio de 1993: PROCESADO
 Resultado del  jueves 29 de j

 Resultado del  sábado 25 de abril de 1992: PROCESADO
 Resultado del  jueves 23 de abril de 1992: PROCESADO
 Resultado del  sábado 18 de abril de 1992: PROCESADO
 Resultado del  jueves 16 de abril de 1992: PROCESADO
 Resultado del  sábado 11 de abril de 1992: PROCESADO
 Resultado del  jueves 9 de abril de 1992: PROCESADO
 Resultado del  sábado 4 de abril de 1992: PROCESADO
 Resultado del  jueves 2 de abril de 1992: PROCESADO
 Resultado del  sábado 28 de marzo de 1992: PROCESADO
 Resultado del  jueves 26 de marzo de 1992: PROCESADO
 Resultado del  sábado 21 de marzo de 1992: PROCESADO
 Resultado del  jueves 19 de marzo de 1992: PROCESADO
 Resultado del  sábado 14 de marzo de 1992: PROCESADO
 Resultado del  jueves 12 de marzo de 1992: PROCESADO
 Resultado del  sábado 7 de marzo de 1992: PROCESADO
 Resultado del  jueves 5 de marzo de 1992: PROCESADO
 Resultado del  sábado 29 de febrero de 1992: PROCESADO
 Resultado del  jueves 27 de febrero de 1992: PROCESADO
 Resultado del  sáb